In [ ]:
import pandas as pd
import numpy as np
eps = np.finfo(float).eps
from numpy import log2 as log




In [ ]:
df = pd.read_excel('Decision_tree_DATA.xlsx')
df.drop(columns =['Temperature Numeric','Humidity Numeric'],inplace=True)

In [ ]:
df


,Outlook,Temperature Nominal,Humidity Nominal,Windy,Play
0,overcast,hot,high,False,yes
1,overcast,cool,normal,True,yes
2,overcast,mild,high,True,yes
3,overcast,hot,normal,False,yes
4,rainy,mild,high,False,yes
5,rainy,cool,normal,False,yes
6,rainy,cool,normal,True,no
7,rainy,mild,normal,False,yes
8,rainy,mild,high,True,no
9,sunny,hot,high,False,no


In [ ]:
def find_entropy(df):
    Target = df.columns[-1]   
    entropy = 0
    values = df[Target].unique()
    for value in values:
        fraction = df[Target].value_counts()[value]/len(df[Target])
        entropy += -fraction*np.log2(fraction)
    return entropy

In [ ]:
def find_entropy_attribute(df,attribute):
    Target = df.columns[-1]   
    target_variables = df[Target].unique()  #This gives all 'Yes' and 'No'
    variables = df[attribute].unique()    #This gives different features in that attribute (like 'Hot','Cold' in Temperature)
    entropy2 = 0
    for variable in variables:
        entropy = 0
        for target_variable in target_variables:
            num = len(df[attribute][df[attribute]==variable][df[Target] ==target_variable])    #eg total sunny yes's/no's
            den = len(df[attribute][df[attribute]==variable])   #eg total sunny 
            fraction = num/(den+eps)  ##what is eps #total (yes's and no's  / total sunny)
            entropy += -fraction*log(fraction+eps)  
        fraction2 = den/len(df)    #eg total sunny / length of df
        entropy2 += -fraction2*entropy 
    return abs(entropy2)

In [ ]:
def find_winner(df):
    Entropy_att = []
    IG = []
    for key in df.keys()[:-1]:#         Entropy_att.append(find_entropy_attribute(df,key))
        IG.append(find_entropy(df)-find_entropy_attribute(df,key))
    return df.keys()[:-1][np.argmax(IG)] 

In [ ]:
def get_subtable(df, node,value):
    return df[df[node] == value].reset_index(drop=True)

In [ ]:
def buildTree(df,tree=None): 
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name  #Here we build our decision tree  #Get attribute with maximum information gain
    node = find_winner(df)#Get distinct value of that attribute e.g Salary is node and Low,Med and High are values
    attValue = np.unique(df[node])#Create an empty dictionary to create tree    
    if tree is None:                    
        tree={}
        tree[node] = {}#We make loop to construct a tree by calling this function recursively. #In this we check if the subset is pure and stops if it is pure. 
    for value in attValue:
        subtable = get_subtable(df,node,value)
        clValue,counts = np.unique(subtable['Play'],return_counts=True)                        
        if len(counts)==1:#Checking purity of subset
            tree[node][value] = clValue[0]                                                    
        else:        
            tree[node][value] = buildTree(subtable) #Calling the function recursively               
    return tree

In [ ]:
tree= buildTree(df)
import pprint
pprint.pprint(tree)

{'Outlook': {'overcast': 'yes',
             'rainy': {'Windy': {False: 'yes', True: 'no'}},
             'sunny': {'Humidity Nominal': {'high': 'no', 'normal': 'yes'}}}}
